# coffea to cabinetry
Template histograms for statistical analysis are defined implicitly in `cabinetry`.
Below is a partial example:
```yaml
Regions:
  - Name: "Signal_region"
    Variable: "lep_pt"
    Filter: "HT_jets >= 1000"
    Binning: [200, 300, 400, 500, 600]
        
  - Name: "Control_region"
    Variable: "lep_pt"
    Filter: "HT_jets < 1000"
    Binning: [0, 300, 600]

Samples:
  - Name: "Signal"
    Tree: "signal"
    SamplePaths: "prediction.root"
    Weight: "weight"

  - Name: "Background"
    Tree: "background"
    SamplePaths: "prediction.root"
    Weight: "weight"

Systematics:
  - Name: "WeightBasedModeling"
    Up:
      Weight: "weight_up"
    Down:
      Weight: "0.7*weight_down"
    Samples: "Background"
    Type: "NormPlusShape"
```
There are two regions/channels: `Signal_region` and `Control_region`.
For both channels, nominal histograms of the `Signal` and `Background` samples are needed (4 templates).
Additionally, for the `Background` sample a systematic variation `WeightBasedModeling` is needed (4 more templates, 1 up/down in each region).
In total, the above implicitly contains instructions for building 8 different template histograms.

### Current `cabinetry` approach
At the moment, `cabinetry` handles the template building like this:
- loop over all regions
  - loop over all samples
    - determine which systematic variations `[s1, s2, s3, ...]` affect given sample (in given region)
    - loop over nominal template `n` and systematics, send instructions for one template at a time (`[n, s1, ...]`) to a backend
    - receive single template from backend, save to file
    
The backend is currently `uproot`-based ([link](https://github.com/alexander-held/cabinetry/blob/c8668005e899556675b5e646e127908849bfe597/src/cabinetry/contrib/histogram_creation.py#L10-L83)).
The relevant function turns histogram-building instructions
```python
def from_uproot(
    ntuple_paths: List[pathlib.Path],
    pos_in_file: str,
    variable: str,
    bins: np.ndarray,
    weight: Optional[str] = None,
    selection_filter: Optional[str] = None,
) -> Tuple[np.ndarray, np.ndarray]:
```
into a histogram (yields + stat. uncertainties).
 

### Naive coffea port
The same method can be used with `coffea`.
`cabinetry` generates histogram building instructions and sends them off to coffea, one instruction at a time.
A generic processor is used to handle those instructions:

In [1]:
import pathlib
from typing import List, Optional, Tuple

import awkward as ak
from coffea import hist, processor
from coffea.nanoevents import BaseSchema
import numpy as np

In [2]:
class MyProcessor(processor.ProcessorABC):
    def __init__(self, channel_info, template_info):
        if channel_info is None or template_info is None:
            raise ValueError

        # set up accumulator from histogram info
        # channels are in dict and not an axis, since binning per channel can vary
        self._accumulator = processor.dict_accumulator()
        for ch in channel_info:
            self._accumulator.update(
                {
                    ch["name"]: hist.Hist(
                        "events",
                        hist.Cat("dataset", "dataset"),
                        hist.Cat("template", "systematic template"),
                        hist.Bin(ch["variable"], ch["observable_label"], ch["binning"]),
                    )
                }
            )
        self.channel_info = channel_info
        self.template_info = template_info

    @property
    def accumulator(self):
        return self._accumulator

    def process(self, events):
        out = self.accumulator.identity()

        # dataset from metadata
        dataset = events.metadata["dataset"]

        # loop over channels
        for ch in self.channel_info:
            # get relevant info for building histogram from metadata
            channel_name = ch["name"]
            variable = ch["variable"]

            # loop over templates
            for template in self.template_info[dataset]:
                template_name = template["name"]
                selection_filter = template["selection_filter"]

                # apply cuts
                events_cut = events[eval(selection_filter, {}, events)]

                observables = eval(variable, {}, events_cut)

                if template["weight"] is not None:
                    weight_expression = template["weight"]
                    weights = eval(weight_expression, {}, events_cut)
                else:
                    weights = np.ones(len(observables))

                out[channel_name].fill(
                    dataset=dataset,
                    template=template_name,
                    weight=weights,
                    **{variable: observables},
                )
        return out

    def postprocess(self, accumulator):
        return accumulator

This processor is steered by instructions `channel_info, template_info` provided in its constructor.
It is able to process multiple channels and templates at once, but not in any way optimized for this and `cabinetry` currently does not make use of this feature.

The processor can be called similarly to the `uproot` backend.
The same information is provided, and converted into dictionaries to hand to the processor constructor.
Note that the `build_single_histogram` signature matches `from_uproot` shown above.

In [3]:
def build_single_histogram(
    ntuple_paths: List[pathlib.Path],
    pos_in_file: str,
    variable: str,
    bins: np.ndarray,
    weight: Optional[str] = None,
    selection_filter: Optional[str] = None,
) -> Tuple[np.ndarray, np.ndarray]:
    # sample can have generic name, not needed here
    # need to convert list of paths to list of strings
    samples = {
        "generic_name": {
            "treename": pos_in_file,
            "files": [str(p) for p in ntuple_paths],
        }
    }

    # template: one at a time, template name not needed
    template_info = {
        "generic_name": [
            {
                "name": "generic_template_name",
                "weight": weight,
                "selection_filter": selection_filter,
            }
        ]
    }

    # channel info: more generic info that is not needed here
    channel_info = [
        {
            "name": "generic_channel_name",
            "variable": variable,
            "observable_label": "generic_label",  # for cosmetics
            "binning": bins,
        }
    ]

    result = processor.run_uproot_job(
        samples,
        None,  # tree name is specified in fileset
        MyProcessor(channel_info, template_info),
        processor.iterative_executor,
        {"schema": BaseSchema},
    )

    yields, variance = result["generic_channel_name"].values(sumw2=True)[
        ("generic_name", "generic_template_name")
    ]
    return yields, np.sqrt(variance)

A lot of information in the dictionaries provided to the `coffea` processor is filled with generic values, since it is not actually relevant for this specific task of building histograms for building a statistical model.
I expect that the design of this interface will evolve over time.

### Optimizing for performance
Right now `cabinetry` creates instructions for building a single histogram, sends those to a backend, waits for the creation of the histogram, saves it, and then continues with the next histogram.
To parallelize this, `cabinetry` should instead create a list of all instructions and then send them of to a backend, allowing to process them asynchronously.
After all instructions are processed, the results need to be gathered and saved.

Once we have a list of histogram building instructions, we could imagine a new layer between `cabinetry` and `coffea`.
This layer could analyze the instructions and group them together to allow for efficient processing, considering CPU/memory/disk.

**How should we approach such a layer? Does this seem like the right way forward in general?**

### Open questions
Traditionally, users of `coffea` seem to implement much more in their processors than just simple selections.
They may include complex event reconstructions methods.
Those are not things that fit well into the `cabinetry` configuration, so one may imagine that people interested in using custom processors inherit from some modified processor base class that handles the basic features needed (processing templates from metadata handed to the processor constructor), but also allows them to implement more advanced functionality.

### Simple example
Below a simple example of using `coffea` to build some histograms is shown.
The relevant input data is generated with [a utility included in `cabinetry`](https://github.com/alexander-held/cabinetry/blob/c8668005e899556675b5e646e127908849bfe597/util/create_ntuples.py).
This example processes multiple template histograms in a single processor (nominal `background` histogram as well as a systeamatic variation together).

First, define all information needed by the processor:
- which samples to process
- which template histograms to build per sample
- which channels to consider

In [4]:
samples = {
    "signal": {
        "treename": "signal",
        "files": ["../cabinetry/ntuples/prediction.root"],
    },
    "background": {
        "treename": "background",
        "files": ["../cabinetry/ntuples/prediction.root"],
    },
}

# list of templates per sample, each template specified via dict (name/selection/weight)
# TODO: allow channel-specific systematics
# TODO: could this be integrated with the sample list above (with files etc.)?
#       - https://github.com/CoffeaTeam/coffea/issues/479
# TODO: how to handle systematics that use different files
template_info = {
    "signal": [
        {
            "name": "nominal",
            "weight": "weight",
            "selection_filter": "lep_charge == 1",
        }
    ],
    "background": [
        {
            "name": "nominal",
            "weight": "weight",
            "selection_filter": "lep_charge == 1",
        },
        {
            "name": "WeightBasedModeling",
            "weight": "weight_up",
            "selection_filter": "lep_charge == 1",
        },
    ],
}

# histogram-related info contains channel name, not needed for processing but possibly
# useful for bookkeeping
# list of channels, dicts per channel contain relevant info
channel_info = [
    {
        "name": "Signal_region",
        "variable": "jet_pt",
        "observable_label": "jet $p_T$ [GeV]",  # for cosmetics
        "binning": [200, 300, 400, 500, 600],
    }
]

Then run `coffea`:

In [5]:
result = processor.run_uproot_job(
    samples,
    None,  # tree name is specified in fileset
    MyProcessor(channel_info, template_info),
    processor.iterative_executor,
    {"schema": BaseSchema},
)

Preprocessing:   0%|          | 0/2 [00:00<?, ?file/s]

Processing:   0%|          | 0/2 [00:00<?, ?chunk/s]

And finally read the histogram content:

In [6]:
for ch in channel_info:
    print(f"channel {ch['name']}")
    for ds in template_info.keys():
        print(f"  dataset: {ds}")
        for tem in template_info[ds]:
            template_name = tem["name"]
            print(f"    template: {template_name}")
            yields, variance = result[ch["name"]].values(sumw2=True)[
                (ds, template_name)
            ]
            print(f"      yields: {yields}")
            print(f"      stdev: {np.sqrt(variance)}")

channel Signal_region
  dataset: signal
    template: nominal
      yields: [ 0.          1.58536913 23.6164268  24.54892223]
      stdev: [0.         0.19931166 0.77410713 0.78830503]
  dataset: background
    template: nominal
      yields: [112.73896936 128.62169539  88.10700838  55.24607072]
      stdev: [4.76136678 5.10645036 4.21104367 3.34933335]
    template: WeightBasedModeling
      yields: [ 98.13569365 154.1222757  135.20449815 103.14744392]
      stdev: [4.17025569 6.14088444 6.47920695 6.2581315 ]
